In [82]:
import os
import logging
import time
import psycopg2

import pandas as pd
import numpy as np 
from sqlalchemy import create_engine

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
 

def connect_postgres():
    database = os.environ['POSTGRES_DB']
    user = os.environ['POSTGRES_USER']
    password = os.environ['POSTGRES_PASSWORD']
    host = os.environ['POSTGRES_SERVER']
    port = 5432

    exc, conn, engine = None, None, None

    for _ in range(5):
        try:
            conn = psycopg2.connect(
                database=database, user=user, password=password, host=host, port=port)
        except Exception as e:
            logging.warning("Error connecting to postgres, will retry in 3 sec: %s", e)
            time.sleep(3)
            exc = e
        else:
            logging.info("Connected...")
            logging.info("Everything goes well from Postgres, you're a fu*** pro...")
            
            engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
                user, password, host, port, database
            ))
            break
    else:
        logging.error("Unable to connect to  %s DB", database)
        raise exc
    
    return [conn, engine]


In [83]:
[conn, engine] = connect_postgres()

INFO:root:Connected...
INFO:root:Everything goes well from Postgres, you're a fu*** pro...


In [84]:
df = pd.read_sql("""
    SELECT code, trans_date_id as date, copy.id as books
    FROM public.copytransaction, public.copy, public.student, public.title, public.transactiontype
    WHERE 
        trans_borrower_code = student.id and
        trans_copy_code_id = copy.id and 
        trans_tittle_code_id = title.id and 
        trans_type_id = transactiontype.id and 
        trans_location_code_id = 5 and 
        trans_type_code IN ('ISS', 'REN', 'NON', 'PLOAN')
    GROUP BY code, trans_date_id, copy.id
""", con=conn)

/opt/app/.venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [86]:
df

,code,date,books
0,198224610,20000928,547476
1,198224610,20000928,547477
2,198224610,20001014,547477
3,198224610,20001117,17499
4,198224610,20001120,172235
...,...,...,...
3352753,201880023,20180305,831823
3352754,201880026,20180305,832497
3352755,201880026,20180307,823259
3352756,201880026,20180313,834960


In [87]:
df = df.astype({col: 'int32' for col in df.select_dtypes('int64').columns})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3352758 entries, 0 to 3352757
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   code    int32
 1   date    int32
 2   books   int32
dtypes: int32(3)
memory usage: 38.4 MB


In [88]:
#exploring the data

# checking null values
df.isnull().sum()

code     0
date     0
books    0
dtype: int64

In [89]:
import numpy as np
import math
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

chunk_max_size = 100000
chunks = int(math.ceil(len(df) / chunk_max_size))

rules = []

In [ ]:
for df_chunk in np.array_split(df, chunks):
    df_chunk.books = df_chunk.books.transform(lambda x: [x])    
    library = df_chunk.groupby(['code','date']).sum()['books'].reset_index(drop=True)
    transform_library = encoder.fit(library).transform(library)
    transactions = pd.DataFrame(transform_library, columns=encoder.columns_)
    
    # TODO: understand this very well
    frequent_itemsets = fpgrowth(transactions, min_support=10/len(df_chunk), use_colnames=True)
    if not frequent_itemsets.empty:
        # rule = association_rules(frequent_itemsets, metric="confidence",  min_threshold=0.5)
        rule = association_rules(frequent_itemsets)
        
        if not rule.empty:
            rule["antecedent_len"] = rule["antecedents"].apply(lambda x: len(x))
            association_rule = rule[
                (rule['antecedent_len'] >= 2) &
                (rule['confidence'] >= 0.5) &
                (rule['lift'] >= 1)
            ]
            
            rules.append(association_rule)
            print("Rules identified: ", len(rule))

Rules identified:  3366
Rules identified:  3043
Rules identified:  14865
Rules identified:  94443
Rules identified:  3340
Rules identified:  20738
Rules identified:  10329
Rules identified:  12565
Rules identified:  30854
Rules identified:  8278
Rules identified:  36041
Rules identified:  4905
Rules identified:  2235
Rules identified:  6942
Rules identified:  2017
Rules identified:  6483
Rules identified:  3028
Rules identified:  2916
Rules identified:  1721
Rules identified:  1657


In [ ]:
result = pd.concat(rules).reset_index(drop=True)
display(result)

In [ ]:
# TODO: save association rules to a database